<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# NLP Using the Twitter API: Guided Lab

<br>

---


<img src="https://snag.gy/RNAEgP.jpg" width="600">

### Can we correctly identify which of these two old men tweeted what?

> *Note: this lab is intended to be a guided lab until the independent practice questions.*


## Goals
---

We are going to attempt to classify whether a tweet comes from Trump or Sanders.  This lab involves multiple steps:
- Create a developer account on Twitter
- Create a method to pull a list of tweets from the Twitter API
- Perform proper preprocessing on our text
- Engineer sentiment feature in our dataset using TextBlob
- Explore supervised classification techniques


## Twitter API Developer Registration
---

If you haven't registered a Twitter account yet, this is a requirement in order to have a "developer" account.

[Twitter Rest API](https://dev.twitter.com/rest/public)



## Create an "App"

---

![](https://snag.gy/HPBQbJ.jpg)

Go to Twitter and register an "app" [apps.twitter.com](https://apps.twitter.com/).

> **Note**: For the required website field you can put a placeholder.

After you set up our app, you will only need to reference the cooresponding keys Twitter generates for our app.  These are the keys that we will use with our application to communicate with the Twitter API.

## Install Python Twitter API library

---

Someone was nice enough to build a Python libary for us. It makes pulling tweets simple: we only need to plug in our keys and start collecting data. The library we will be using is provided by [Python Twitter Tools](http://mike.verdone.ca/twitter/).

To install it, just run the next frame (there is no conda package).

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [9]:
# pip install twitter python-twitter

## Some Boring Twitter Rules
---

**Twitter notifies you they will rate limit your requests:**

>When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate limit window, then it allows you to make 15 requests per window — on behalf of your application. This limit is considered completely separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of what Twitter says are "the rules":

![](https://snag.gy/yJ6vIH.jpg)


## About those Keys: OAuth Review
---

![](https://g.twimg.com/dev/documentation/image/appauth_0.png)

## What's going on here?  Take a minute..

## Our Application Keys
---

Take note of your application keys you will use to connect to Twitter and mine tweets from the official Bernie Sanders and Donald Trump twitter accounts:

![](https://snag.gy/H1djQK.jpg)

## `TweetMiner` class structure

---

The following code will get you up and running, providing connectivity to twitter. The class has the ability to make requests and can eventually transform the JSON responses into DataFrames.

This is a great example of using object-oriented Python to organize our code!

> **Note:** "request_limit" is used in this class to limit the number of tweets that are pulled per instance request.  Setting it to something lower until you've worked the bugs out of your request, and captured the data you want, is essential to avoiding the rate limit blocks.

### Twitter API key setup

Fill the information below in with the keys for your account.

- **consumer_key** - Find this in your app page under the "Keys and Access Tokens"
- **consumer_secret** - Right under **consumer_key** in the "Keys and Access Tokens" tab
- **access_token_key** - You will need to click the button to generate tokens to get this
- **access_token_secret** - Also available after you generate tokens


In [10]:
import twitter, re, datetime, pandas as pd

twitter_keys = {
    'consumer_key':        'lA6UQplM5sxIAIr83ueUl9sgE',
    'consumer_secret':     'f94t4BD6Vj7aCVkX1qAIVwsP4x69J2vXvm61lTIuwb9GfmdsuP',
    'access_token_key':    '1203210464-dYE1FvoUx1GjVcoyok3U1brWDpELBJEcSNGC1OC',
    'access_token_secret': 'kuVli1j010RPdYTboz9iIyp8QxX6PqRApdi49baLqBgzo'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret']
)


In [11]:
class TweetMiner(object):

    result_limit    =   20    
    api             =   False
    data            =   []
    
    twitter_keys = {
        'consumer_key':        'KmN03M1X1pImZ43sqdIu4yfnE',
        'consumer_secret':     'ePlIrIX5VXbZnO7DBu1RbFlw5lOai9dQr9n5TZb6vxnIdrr5Fz',
        'access_token_key':    '185036086-Q7K5IjuSoQZJwSIqD0wyHf6t62iPKatmfaPkriAM',
        'access_token_secret': 'cYpQz3xWHQbLplOj8iSeiNSOmMcsTOXmWcKMrJ9buLj5d'
    }
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.api = api
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dyerrington", mine_rewteets=False, max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1)        
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'handle':          item.user.name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                }
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data

## Instantiate the class
---

Make sure you pass the keys dictionary and the api as arguments.

**Check:** call the object's `mine_user_tweets()` method, providing a user to pull the tweets of.

In [12]:
miner = TweetMiner(twitter_keys, api, result_limit=2)

In [13]:
sanders = miner.mine_user_tweets(user="berniesanders", max_pages=5)
donald = miner.mine_user_tweets(user="realDonaldTrump", max_pages=5)

In [14]:
print(sanders[0])

{'tweet_id': 1030108666668740608, 'handle': 'Bernie Sanders', 'retweet_count': 364, 'text': 'RT @AndrewGillum: This Friday is a big day for our campaign as @BernieSanders comes to the Sunshine State for two rallies. RT if we’ll see…', 'mined_at': datetime.datetime(2018, 8, 16, 16, 45, 10, 473991), 'created_at': 'Thu Aug 16 15:06:59 +0000 2018'}


In [15]:
print(donald[0])

{'tweet_id': 1030116078125568006, 'handle': 'Donald J. Trump', 'retweet_count': 2700, 'text': 'The Queen of Soul, Aretha Franklin, is dead. She was a great woman, with a wonderful gift from God, her voice. She will be missed!', 'mined_at': datetime.datetime(2018, 8, 16, 16, 45, 11, 896349), 'created_at': 'Thu Aug 16 15:36:26 +0000 2018'}


### Convert the tweet ouputs to a pandas DataFrame

> *Hint: this is as easy as passing it to the DataFrame constructor!*

In [16]:
pd.DataFrame(sanders).head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Thu Aug 16 15:06:59 +0000 2018,Bernie Sanders,2018-08-16 16:45:10.473991,364,RT @AndrewGillum: This Friday is a big day for...,1030108666668740608
1,Thu Aug 16 15:06:23 +0000 2018,Bernie Sanders,2018-08-16 16:45:10.473999,38,Congratulations to Peter Welch who won his pri...,1030108517074710528
2,Thu Aug 16 15:05:29 +0000 2018,Bernie Sanders,2018-08-16 16:45:10.727996,228,I am very thankful to everyone who voted for m...,1030108289298784256
3,Thu Aug 16 01:07:44 +0000 2018,Bernie Sanders,2018-08-16 16:45:10.728005,1813,Many of the ideas we’ve been talking about for...,1029897462289510400
4,Thu Aug 16 01:05:48 +0000 2018,Bernie Sanders,2018-08-16 16:45:10.973448,303,"RT @CBSThisMorning: ""All across this country, ...",1029896975276302336


##  Create the training data

---

Let's get our "mined" data from the Twitter API.  

1. Mine Trump tweets
- Create a tweet DataFrame
- Mine Sanders tweets
- Append the results to our DataFrame

In [37]:
# we only need to "instantiate" once.  Then we can call mine_user_tweets as much as we want.
miner = TweetMiner(twitter_keys, api, result_limit=400)
trump_tweets = miner.mine_user_tweets("realDonaldTrump")

In [38]:
trump_df = pd.DataFrame(trump_tweets)
print trump_df.shape

(995, 6)


In [39]:
bernie_tweets = miner.mine_user_tweets('berniesanders')

In [40]:
bernie_df = pd.DataFrame(bernie_tweets) 
print bernie_df.shape

(1000, 6)


In [41]:
tweets = pd.concat([trump_df, bernie_df], axis=0)
tweets.shape

(1995, 6)

## Any interesting ngrams going on with Trump?
---

Set up a vectorizer from sklearn and fit the text of Trump's tweets with an ngram range from 2 to 4. Figure out what the most common ngrams are.

> **Note:** It's up to you whether you want to remove stopwords or not. How does keeping or removing stopwords affect the results?

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'https co', 454),
 (u'of the', 83),
 (u'will be', 63),
 (u'in the', 62),
 (u'to the', 57),
 (u'thank you', 55),
 (u'for the', 49),
 (u'tax cuts', 42),
 (u'fake news', 42),
 (u'our country', 37),
 (u'at the', 36),
 (u'to be', 36),
 (u'we are', 30),
 (u'our great', 30),
 (u'of our', 28),
 (u'and the', 28),
 (u'puerto rico', 26),
 (u'we will', 25),
 (u'honor to', 25),
 (u'the great', 23)]

### Look at the ngrams for Bernie Sanders

In [43]:
# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(bernie_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'https co', 428),
 (u'health care', 139),
 (u'of the', 66),
 (u'bernie sanders', 53),
 (u'in the', 49),
 (u'to the', 44),
 (u'for the', 43),
 (u'we must', 43),
 (u'we need', 39),
 (u'we are', 37),
 (u'is not', 36),
 (u'for all', 35),
 (u'millions of', 35),
 (u'donald trump', 33),
 (u'going to', 33),
 (u'is the', 31),
 (u'this country', 30),
 (u'on the', 30),
 (u'should be', 28),
 (u'and the', 28)]

## Processing the tweets and building a model

---

To do classfication we will need to convert the tweets into a set of features.

**You will need to:**
- Vectorize input text data.
- Intialize a model (try Logistic regression).
- Train / Predict / cross-validate.
- Evaluate the performance of the model.

> **Bonus:** you may have noticed that there are website links in the tweets. What additional preprocessing steps can you do before building the model?


In [44]:
# BONUS
# Using the textacy package to do some more comprehensive preprocessing
# http://textacy.readthedocs.io/en/latest/
from textacy.preprocess import preprocess_text

tweet_text = tweets['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

In [45]:
print tweet_text[0:3]

[ u'Across the battlefields, oceans, and harrowing skies of Europe and the Pacific throughout the war, one great battle\u2026 https://t.co/XelKHx4Ibo'
 u'Today, as we Remember Pearl Harbor, it was an incredible honor to be joined with surviving Veterans of the attack o\u2026 https://t.co/c4ZYnXSAuj'
 u'Today, the U.S. flag flies at half-staff at the @WhiteHouse, in honor of National Pearl Harbor Remembrance Day.\u2026 https://t.co/561NBdqSsb']


In [46]:
print clean_text[0:3]

[u'across the battlefields oceans and harrowing skies of europe and the pacific throughout the war one great battle url', u'today as we remember pearl harbor it was an incredible honor to be joined with surviving veterans of the attack o url', u'today the us flag flies at halfstaff at the whitehouse in honor of national pearl harbor remembrance day url']


In [50]:
# target is the handle.
# make trump 1 and sanders 0
y = tweets['handle'].map(lambda x: 1 if x == 'Donald J. Trump' else 0).values
print np.mean(y)

0.498746867168


In [51]:
from sklearn.linear_model import LogisticRegression

# Preprocess our text data to Tfidf
tfv = TfidfVectorizer(ngram_range=(1,4), max_features=2000)
X = tfv.fit_transform(clean_text).todense()
print X.shape

(1995, 2000)


In [52]:
# cross-validate the accuracy:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(LogisticRegression(), X, y, cv=10)

import numpy as np
print accuracies
print np.mean(accuracies)

# Setup logistic regression (or try another classification method here)
estimator = LogisticRegression()
estimator.fit(X, y)


[ 0.85        0.895       0.87        0.905       0.88        0.88944724
  0.91457286  0.89447236  0.87437186  0.85929648]
0.883216080402


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [53]:
# Very good accuracy considering the baseline is 50%

## Check the predicted probability for a random Sanders and Trump tweet
---

Below are provided a couple of tweets from both Sanders and Trump. I'm sure you can figure out on your own which one is which.

Estimate the predicted probability of being trump for the two tweets.

In [54]:
# Prep our source as TfIdf vectors
source_test = [
    "Demanding that the wealthy and the powerful start paying their fair share of taxes that's exactly what the American people want.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!"
]

############
# NOTE:  Do not re-initialize the tfidf vectorizor or the feature space willbe overwritten and
# hence your transform will not match the number of features you trained your model on.
#
# This is why you only need to "transform" since you already "fit" previously
#
####

Xtest = tfv.transform(source_test)

# Predict using previously trained logist regression `estimator`
estimator.predict_proba(Xtest)


array([[ 0.77741316,  0.22258684],
       [ 0.33131595,  0.66868405]])

In [55]:
# The 1st column is probability of being Bernie, and 2nd Trump. The classifier is getting it right.

## Independent practice questions

---

### 1. Pull tweets for some new users.

Experiment with using more data.  The API will not like it if you blow through their limits - be careful.  Try to grab only what you need one time, then work on the copy of the objects that are returned.  

> Read the documentation about rate limits and see if you can get enough without hitting the rate limit.  Are there any options available in the API to avoid such a problem?

**Pull tweets for more than two different users of your choice.**

In [56]:
# We deviate from trump / sanders using student tweets here to illustrate the NLP pipeine with twitter data

twitter_handles = ["dril", "LaziestCanine",'ch000ch']
tweets = {}

for twitter_handle in twitter_handles:
    print "Mining tweets for: ", twitter_handle
    miner = TweetMiner(twitter_keys, api, result_limit=500)
    tweets[twitter_handle] = miner.mine_user_tweets(user=twitter_handle, max_pages=10)


Mining tweets for:  dril
Mining tweets for:  LaziestCanine
Mining tweets for:  ch000ch


In [57]:
multi = pd.DataFrame(tweets['dril'])
multi = multi.append(pd.DataFrame(tweets['LaziestCanine']))
multi = multi.append(pd.DataFrame(tweets['ch000ch']))

print multi.shape


(5992, 6)


In [58]:
multi.handle.value_counts()

Lazy dog    1998
wint        1998
chuuch      1996
Name: handle, dtype: int64

### 2. Build a multi-class classification model to distinguish between the users.

Try a new type of model than we used before.

In [59]:
tweet_text = multi['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

y = multi['handle'].map(lambda x: 0 if x == 'wint' else 1 if x == 'Lazy dog' else 2).values

In [60]:
tfv = TfidfVectorizer(ngram_range=(1,3), max_features=2500)
X = tfv.fit_transform(clean_text)

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

rf = RandomForestClassifier(n_estimators=250, verbose=1)
knn = KNeighborsClassifier(n_neighbors=7)

rf.fit(X_train, y_train)
knn.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    4.8s finished


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')

In [63]:
# Random forest score:
print 'RF:', rf.score(X_test, y_test)
print 'KNN:', knn.score(X_test, y_test)

RF:

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    0.4s finished


 0.70467185762
KNN: 0.38765294772


In [64]:
# Baseline score:
multi.handle.value_counts()/multi.shape[0]

Lazy dog    0.333445
wint        0.333445
chuuch      0.333111
Name: handle, dtype: float64

In [65]:
rf_yhat = knn.predict(X_test)

### 3. Make a confusion matrix and classification report.

In [66]:
from sklearn.metrics import classification_report, confusion_matrix

print classification_report(y_test, rf_yhat)


             precision    recall  f1-score   support

          0       0.56      0.17      0.26       636
          1       0.35      0.77      0.48       584
          2       0.46      0.24      0.32       578

avg / total       0.46      0.39      0.35      1798



In [67]:
# Confusion Matrix
print confusion_matrix(y_test, rf_yhat)

[[107 437  92]
 [ 60 450  74]
 [ 24 414 140]]


### 4. What is the most and least "distinctive" tweets for each user?

To find this, identify the tweet that has the highest (correct) predicted probability of being that user's tweet for each user.

In [68]:
rf.fit(X, y)
pp = rf.predict_proba(X)

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    7.7s finished
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    1.1s finished


In [69]:
pp[0:5]

array([[ 0.868     ,  0.072     ,  0.06      ],
       [ 0.69568698,  0.27122401,  0.03308902],
       [ 0.8166383 ,  0.08195745,  0.10140426],
       [ 0.892     ,  0.024     ,  0.084     ],
       [ 0.984     ,  0.004     ,  0.012     ]])

In [70]:
pp = pd.DataFrame(pp, columns=['dril_pp', 'laziestcanine_pp', 'ch000ch_pp'])

In [71]:
print multi.shape, pp.shape

(5992, 6) (5992, 3)


In [72]:
tweets_pp = pd.concat([multi.reset_index(), pp.reset_index()], axis=1)
tweets_pp.head(2)

,index,created_at,handle,mined_at,retweet_count,text,tweet_id,index,dril_pp,laziestcanine_pp,ch000ch_pp
0,0,Wed Dec 06 17:04:53 +0000 2017,wint,2017-12-08 00:06:17.473959,2211,"just had the check Mark for 6 mins, but lost i...",938454206683508739,0,0.868000,0.072000,0.060000
1,1,Wed Dec 06 07:52:57 +0000 2017,wint,2017-12-08 00:06:17.473971,2069,Can i citizens arrest girls,938315310230310912,1,0.695687,0.271224,0.033089


In [73]:
print 'Most dril:', tweets_pp[tweets_pp.handle == 'wint'].sort_values('dril_pp', ascending=False).text.values[0]
print 'Least dril:', tweets_pp[tweets_pp.handle == 'wint'].sort_values('dril_pp', ascending=True).text.values[0]

Most dril: @HarshilShah1910 shut the fuck up
Least dril: @antonwheel thanks


In [74]:
print 'Most LaziestCanine:', tweets_pp[tweets_pp.handle == 'Lazy dog'].sort_values('laziestcanine_pp', ascending=False).text.values[0]
print 'Least LaziestCanine:', tweets_pp[tweets_pp.handle == 'Lazy dog'].sort_values('laziestcanine_pp', ascending=True).text.values[0]

Most LaziestCanine: RT @LaziestCanine: bros, y'all letting ur girl go out lookin like this? https://t.co/YcApnX1Inn
Least LaziestCanine: @Jaclyn_Mariee_ lmao tru


In [75]:
print 'Most chuuch:', tweets_pp[tweets_pp.handle == 'chuuch'].sort_values('ch000ch_pp', ascending=False).text.values[0]
print 'Least chuuch:', tweets_pp[tweets_pp.handle == 'chuuch'].sort_values('ch000ch_pp', ascending=True).text.values[0]

Most chuuch: @dubstep4dads gains baby gains!!!!! 😩💪💪💪
Least chuuch: https://t.co/IIfFmqEyNN
